# 情感分析代码（情感词典与SVM）
#### 备注：需要将 directory 变量更改成存放数据集的文件夹

### 1. 情感词典法

In [1]:
import jieba
import jieba.analyse
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from gensim.models.word2vec import Word2Vec
from sklearn.svm import SVC

In [2]:
# 导入并构建情感词典
directory = "C:\\Users\\keehu\\Downloads\\chinese_sentiment_dictionary-master\\file\\情感词典\\知网\\"
nlp_dict_path = {"pos_dict" : directory + "正面评价词语（中文）.txt",
                 "neg_dict" : directory + "负面评价词语（中文）.txt"      
                }

nlp_dict = {}

for name, path in nlp_dict_path.items():
    curr_set = set()
    with open(path) as f:
        count = f.readline().split("\t")[1]
        f.readline()
        while f:
            line = f.readline().strip()
            if line == "":
                break
            curr_set.add(line)
    nlp_dict[name] = curr_set

In [3]:
# 导入程度词词典
level_dict_path = directory + "程度级别词语（中文）.txt"
level_dict = dict()
level = [3,2,1.5,0.5,0.5,3,-1]
with open(level_dict_path) as f:
    count = f.readline().split("\t")[1]
    f.readline()
    i = 0
    while i <= 7:
        line = f.readline().strip()
        if len(line.split()) > 1:
            continue
        if line == "":
            i += 1
        else:
            level_dict[line] = level[i]

In [4]:
# 导入停用词词典
stopwords = [line.strip() for line in open(directory + "stoplist1.txt",'r', encoding = "utf-8").readlines()]

In [5]:
# 分词
def split_sentence(sentence):
    global stopwords
    seg_list = jieba.lcut(sentence, cut_all=False)
    outputstr = []
    for word in seg_list:
        if word not in stopwords:
            outputstr.append(word)
    return outputstr

In [6]:
# 寻找程度词并加权
def search_level(words, score, j, level_p, level_dict, detected):
    if j > level_p:
        while j < len(words) and words[j] in level_dict:
            score *= level_dict[words[j]]
            level_p = j
            detected.append(words[j])
            j += 1
    return level_p, score

# 计算情感分数
def sentiment_score(sentence, dict_set, level_dict):
    words = split_sentence(sentence)
    words.reverse()
    score_stack = []
    level_p = -1
    total = 0
    detected = []
    i = 0
    while i < len(words):
        if words[i] in dict_set["pos_dict"]:
            detected.append(words[i])
            score = 1
            level_p, score = search_level(words, score, i - 1, level_p, level_dict, detected)
            level_p, score = search_level(words, score, i + 1, level_p, level_dict, detected)
            total += score
            
        elif words[i] in dict_set["neg_dict"]:
            detected.append(words[i])
            score = -1
            level_p, score = search_level(words, score, i - 1, level_p, level_dict, detected)
            level_p, score = search_level(words, score, i + 1, level_p, level_dict, detected)
            total += score
        i += 1
            
    return (total, 1 if total > 0 else 0, detected)

In [7]:
# 读入数据集
df = pd.read_csv(directory + "online_shopping_10_cats.csv", encoding="utf-8")
pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', None)
print("数据行数与列数: "+str(df.shape))

数据行数与列数: (62774, 3)


In [8]:
# 将停用词加入模型
jieba.analyse.set_stop_words(directory + "stoplist.txt")
jieba.suggest_freq(["无语"], True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\keehu\AppData\Local\Temp\jieba.cache
Loading model cost 0.543 seconds.
Prefix dict has been built successfully.


3

In [9]:
# 查看数据
df.head()

,cat,label,review
0,书籍,1,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一颗年轻的心。我想，这是他能很好的和孩子沟通的一个重要因素。读刘墉的文章，总能让我看到一个快乐的平易近人的父亲，他始终站在和孩子同样的高度，给孩子创造着一个充满爱和自由的生活环境。很喜欢刘墉在字里行间流露出的做父母的那种小狡黠，让人总是忍俊不禁，父母和子女之间有时候也是一种战斗，武力争斗过于低级了，智力较量才更有趣味。所以，做父母的得加把劲了，老思想老观念注定会一败涂地，生命不息，学习不止。家庭教育，真的是乐在其中。
1,书籍,1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理的火花。整本书的结构颇有特点，从当时（本书写于八十年代）流行的计算机话题引入，再用数学、物理学、宇宙学做必要的铺垫——这些内容占据了大部分篇幅，最后回到关键问题：电脑能不能代替人脑。和现在流行的观点相反，作者认为人的某种“洞察”是不能被算法模拟的。也许作者想说，人的灵魂是无可取代的。
2,书籍,1,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？为什么在文化上有着深刻纽带关系的中国和日本却在经济发展上有着极大的差异？为什么英国的北美殖民地造就了经济强大的美国，而西班牙的北美殖民却造就了范后的墨西哥？……很有价值，但不包括【中国近代史专业】。
3,书籍,1,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延续，没有战后的民发反思，没有～，就不会让日本成为一个经济强国．当然，美国人也给日本人带来了耻辱．对日中关系也造成了深远的影响．文中揭露了＂东京审判＂中很多鲜为人知的东西．让人惊醒．唉！中国人民对日本的了解是不是太少了．
4,书籍,1,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵的有两点，一是他的理科知识不错，虽不能媲及罗素，但与理科知识很差的作家相比，他的文章可读性要强；其二是他人格和文风的朴实，不造作，不买弄，让人喜欢。读他的作品，犹如听一个好友和你谈心，常常唤起心中的强烈的共鸣。他的作品90年后的更好些。衷心祝愿周国平健康快乐，为世人写出更多好作品。


In [10]:
# 观察数据类别
df["cat"].unique()

array(['书籍', '平板', '手机', '水果', '洗发水', '热水器', '蒙牛', '衣服', '计算机', '酒店'],
      dtype=object)

In [11]:
# 预处理与分词
df["review"] = df["review"].astype(str)
df["splitted"] = [split_sentence(i) for i in df["review"]]

In [12]:
# 进行情感得分的计算
pair = [sentiment_score(i, nlp_dict, level_dict) for i in df["review"]]

In [13]:
df["score"] = [i[0] for i in pair]
df["sentiment"] = [i[1] for i in pair]
df["detected"] = [i[2] for i in pair]

In [14]:
# 输出混淆矩阵
confusion_matrix(df["label"], df["sentiment"])

array([[25831,  5246],
       [ 4462, 27235]], dtype=int64)

In [15]:
# 输出预测精确率与召回率
print(classification_report(df["label"], df["sentiment"]))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84     31077
           1       0.84      0.86      0.85     31697

    accuracy                           0.85     62774
   macro avg       0.85      0.85      0.85     62774
weighted avg       0.85      0.85      0.85     62774



In [16]:
# 观察各个类别的预测精确率
arr = []
for i in df.cat.unique():
    wrong = df[(df.label != df.sentiment) & (df.cat == i)]
    total = df[(df.cat == i)].size
    print(i, 1 - wrong.size / total)
    arr.append(wrong)

书籍 0.7592833030381719
平板 0.8588
手机 0.87343951786483
水果 0.8566
洗发水 0.8495
热水器 0.8017391304347826
蒙牛 0.7919331037875061
衣服 0.8996
计算机 0.8081162324649298
酒店 0.8170999999999999


### 2. SVM模型法

In [17]:
# 将数据集分成训练集和测试集
neg= df[df.label==0]["splitted"]
pos= df[df.label==1]["splitted"]
y=np.concatenate((np.ones(len(pos)),np.zeros(len(neg))))
x_train,x_test,y_train,y_test=train_test_split(np.concatenate((pos, neg)),y,test_size=0.3)

In [18]:
# 构建句子向量
def build_sentence_vector(text,size,w2v_model):
    vec=np.zeros(size).reshape((1,size))
    count=0
    for word in text:
        try:
            vec+=w2v_model.wv[word].reshape((1,size))
            count+=1
        except KeyError:
            continue
    if count!=0:
        vec/=count
    return vec
 
# 计算词向量
def get_train_vecs(x_train, x_test):
    n_dim=300 
    
    w2v_model=Word2Vec(vector_size=n_dim, window=5, sg=0, hs=0, negative=5, min_count=10)
    w2v_model.build_vocab(x_train) 
    w2v_model.train(x_train, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs) 
 
    train_vecs=np.concatenate([build_sentence_vector(z,n_dim,w2v_model) for z in x_train])
    
    w2v_model.train(x_test,total_examples=w2v_model.corpus_count,epochs=w2v_model.epochs)
    test_vecs=np.concatenate([build_sentence_vector(z,n_dim,w2v_model) for z in x_test])
    return train_vecs, test_vecs, w2v_model

In [19]:
# 构建SVM模型与预测
train_vecs, test_vecs, w2v_model = get_train_vecs(x_train, x_test)
clf = SVC(kernel='rbf',verbose=True)
clf.fit(train_vecs,y_train)
test_pred = clf.predict(test_vecs)

[LibSVM]

In [20]:
# 输出混淆矩阵
confusion_matrix(y_test, test_pred)

array([[8725,  541],
       [1294, 8273]], dtype=int64)

In [21]:
# 输出测试集预测精确率与召回率
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90      9266
         1.0       0.94      0.86      0.90      9567

    accuracy                           0.90     18833
   macro avg       0.90      0.90      0.90     18833
weighted avg       0.91      0.90      0.90     18833

